Dans ce TP, nous considérons des trajets en vélo partagé (similaire au Vélib) en Californie. Deux jeux de données sont fournis : l'un qui contient les stations de vélo, l'autre, les trajets à vélo. Les déplacements à vélo se font d'une station à l'autre.

In [ ]:
from pathlib import Path
import sqlite3

import pandas as pd

In [53]:
DATA_DIR = Path("../../data")
station_file_path = DATA_DIR / "stationData.csv"
trip_file_path = DATA_DIR / "tripData.csv"

Charger le fichier stationData.csv dans un DataFrame `station_df` et le fichier tripData.csv dans un DataFrame `trip_df`. Pour chaque DataFrame, il vous faudra demander une inférence des schémas et indiquer que la première ligne est un en-tête.

In [54]:
station_df = pd.read_csv(station_file_path)
trip_df = pd.read_csv(trip_file_path)

Afficher les schémas/informations des deux DataFrames.

In [55]:
print(station_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_id    70 non-null     int64  
 1   name          70 non-null     object 
 2   lat           70 non-null     float64
 3   long          70 non-null     float64
 4   dockcount     70 non-null     int64  
 5   landmark      70 non-null     object 
 6   installation  70 non-null     object 
dtypes: float64(2), int64(2), object(3)
memory usage: 4.0+ KB
None


In [56]:
print(trip_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354152 entries, 0 to 354151
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   TripID          354152 non-null  int64 
 1   Duration        354152 non-null  int64 
 2   StartDate       354152 non-null  object
 3   StartStation    354152 non-null  object
 4   StartTerminal   354152 non-null  int64 
 5   EndDate         354152 non-null  object
 6   EndStation      354152 non-null  object
 7   EndTerminal     354152 non-null  int64 
 8   Bike#           354152 non-null  int64 
 9   SubscriberType  354152 non-null  object
 10  ZipCode         353874 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB
None


In [57]:
station_df.head()

,station_id,name,lat,long,dockcount,landmark,installation
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


In [58]:
station_df["dockcount"].describe()

count    70.000000
mean     17.657143
std       4.010442
min      11.000000
25%      15.000000
50%      15.000000
75%      19.000000
max      27.000000
Name: dockcount, dtype: float64

In [59]:
trip_df.head()

,TripID,Duration,StartDate,StartStation,StartTerminal,EndDate,EndStation,EndTerminal,Bike#,SubscriberType,ZipCode
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
1,913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
2,913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
3,913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
4,913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069


Créer une base SQLite ainsi qu'une vue SQL pour chaque DataFrame.

In [60]:
conn = sqlite3.connect(":memory:")
station_df.to_sql("station", conn, index=False, if_exists="replace")
trip_df.to_sql("trip", conn, index=False, if_exists="replace")

354152

Trouver deux façons de calculer le nombre de trajets, l'une en appelant une méthode sur `trip_df` directement, l'autre en rédigeant une requête SQL sur la vue correspondante.

In [61]:
# Méthode pandas
len(trip_df)

354152

In [62]:
# Méthode SQL
query_1 = "SELECT COUNT(*) AS trip_count FROM trip"
trip_count = pd.read_sql(query_1, conn)
print(trip_count.iloc[0]["trip_count"])

354152


Écrire une requête permettant de compter le nombre de trajets qui démarrent et se terminent à la même station.

In [64]:
# import re
# regex = ...
# trip_df["StartTerminal"].apply(lambda x: re.la_fonction_de_r_qui_cherche_regex_dans_x(x, regex))

In [65]:
# Méthode pandas
mask = trip_df["StartTerminal"] == trip_df["EndTerminal"]
print(f"Nombre de trajets circulaires : {len(trip_df[mask])}.")

Nombre de trajets circulaires : 10276.


In [66]:
# Méthode SQL
query_2 = """
SELECT COUNT(*)
FROM TRIP
WHERE StartTerminal = EndTerminal
"""
circular_trips = pd.read_sql(query_2, conn)
print(circular_trips)

   COUNT(*)
0     10276


Obtenir l’id des stations associées à ces trajets. Écrire une requête renvoyant la liste des terminaux concernés ainsi que le nombre de trajets pour chacun de ces terminaux. Trier le résultat par ordre décroissant de nombre de trajets.

In [67]:
# Méthode SQL
query_3 = """
SELECT StartTerminal AS terminal, COUNT(*) AS trip_count
FROM trip
WHERE StartTerminal = EndTerminal
GROUP BY terminal
ORDER BY trip_count DESC
"""
print(pd.read_sql(query_3, conn))

    terminal  trip_count
0         60         850
1         50         708
2         35         348
3         76         320
4         74         307
..       ...         ...
65        80          26
66        24          23
67        83          21
68        21          12
69        26           7

[70 rows x 2 columns]


In [70]:
# Méthode pandas
# A : appliquer la fonction d'agrégation,
# B : filtrer,
# C : créer les groupes de données,
# D : trier par valeurs décroissantes.
# B > C > A > D

trip_df[
    trip_df["StartTerminal"] == trip_df["EndTerminal"]
].groupby(
    "StartTerminal"
).size(
).reset_index(
    name="trip_count"
).sort_values(
    by="trip_count", ascending=False
)

,StartTerminal,trip_count
48,60,850
40,50,708
28,35,348
64,76,320
62,74,307
...,...,...
66,80,26
17,24,23
68,83,21
14,21,12


Compléter le résultat en indiquant le nombre de docks (dockcount) des stations concernées.

In [ ]:
# Méthode pandas

trip_df[
    trip_df["StartTerminal"] == trip_df["EndTerminal"]
].groupby(
    "StartStation"
).size(
).reset_index(
    name="trip_count"
).sort_values(
    by="trip_count", ascending=False
)

In [69]:
# Méthode SQL
query_f = """
SELECT trip.StartTerminal AS terminal, station.dockcount, COUNT(*) as trip_count
FROM trip
JOIN station ON trip.StartTerminal = station.station_id
WHERE trip.StartTerminal = trip.EndTerminal
GROUP BY terminal, station.dockcount
ORDER BY trip_count DESC
"""

print(pd.read_sql(query_f, conn))

    terminal  dockcount  trip_count
0         60         15         850
1         50         23         708
2         35         11         348
3         76         19         320
4         74         23         307
..       ...        ...         ...
65        80         15          26
66        24         15          23
67        83         15          21
68        21         15          12
69        26         15           7

[70 rows x 3 columns]
